In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from data_utils import create_date_df

size = 100
ar_params = [0.7]
ma_params = [0.4, -0.2]
residuals = np.random.normal(0, 1, size)

# ARIMA(1, 2) == AR(1) + MA(2)
data = [0, 0]
for i in range(2, size):
    # AR(1) (I(1)): 자기 자신의 직전 항 값을 통해 예측값 생성
    ar_term = ar_params[0] * (data[i-1] - data[i-2])
    # MA(2) : 현재 항 이전의 2개 항의 오차값을 통해 예측값 생성
    # I(1)은 해당 항에 영향을 끼치지 않음 -> data[] 형태로 표현되는 데이터값과 별개 항이기 때문
    ma_term = ma_params[0] * residuals[i-1] + ma_params[1] * residuals[i-2]
    # 예측값을 data에 밀어넣음
    # 위의 ar_term + ma_term으로 계산되는 값이 현재 항의 차분이기 때문에
    # data[i] - data[i-1] = ar_term + ma_term + residuals[i] 형태가 됨
    # 따라서 현재 추가해야 할 data[i] = data[i-1] + ar_term + ma_term + residuals[i]
    data.append(data[-1] + (ar_term + ma_term + residuals[i]))

df = create_date_df(size, data)
print(df.head(10))

        Date     Value
0 2023-01-01  0.000000
1 2023-01-02  0.000000
2 2023-01-03  1.278922
3 2023-01-04  3.197743
4 2023-01-05  1.858207
5 2023-01-06 -0.943887
6 2023-01-07 -1.558164
7 2023-01-08  0.243265
8 2023-01-09  2.046011
9 2023-01-10  2.081751


In [2]:
from statsmodels.tsa.arima.model import ARIMA

model = ARIMA(df['Value'], order=(1, 1, 2)) # 차분의 차수를 1로 설정한 ARIMA
results = model.fit()
print(results.summary())

                               SARIMAX Results                                
Dep. Variable:                  Value   No. Observations:                  100
Model:                 ARIMA(1, 1, 2)   Log Likelihood                -172.575
Date:                Fri, 18 Aug 2023   AIC                            353.150
Time:                        16:22:33   BIC                            363.530
Sample:                             0   HQIC                           357.350
                                - 100                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.2811      0.143      1.968      0.049       0.001       0.561
ma.L1         -0.1768      8.929     -0.020      0.984     -17.677      17.323
ma.L2         -0.8229      7.328     -0.112      0.9

C:\Users\김동현\AppData\Roaming\Python\Python311\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
